In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# CUSTOMIZABLE: Collect/Prepare data
datapoint_size = 1000
batch_size = 1000
steps = 10000
actual_W1 = 2
actual_W2 = 5
actual_b = 7 
learn_rate = 0.001
log_file = "/tmp/feature_2_batch_1000"

In [3]:
# Model linear regression y = Wx + b
x = tf.placeholder(tf.float32, [None, 2], name="x")
W = tf.Variable(tf.zeros([2,1]), name="W")
b = tf.Variable(tf.zeros([1]), name="b")

with tf.name_scope("Wx_b") as scope:
    product = tf.matmul(x,W)
    y = product + b

In [4]:
# Add summary ops to collect data
W_hist = tf.summary.histogram("weights", W)
b_hist = tf.summary.histogram("biases", b)
y_hist = tf.summary.histogram("y", y)

y_ = tf.placeholder(tf.float32, [None, 1])

In [5]:
# Cost function sum((y_-y)**2)
with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(tf.square(y_-y))
    cost_sum = tf.summary.scalar("cost", cost)

# Training using Gradient Descent to minimize cost
with tf.name_scope("train") as scope:
    train_step = tf.train.GradientDescentOptimizer(learn_rate).minimize(cost)

In [6]:
all_xs = []
all_ys = []
for i in range(datapoint_size):
    # Create fake data for y = 2.x_1 + 5.x_2 + 7
    
    x_1 = i%10
    x_2 = np.random.randint(datapoint_size/2)%10
    y = actual_W1 * x_1 + actual_W2 * x_2 + actual_b
    
    # Create fake data for y = W.x + b where W = [2, 5], b = 7
    all_xs.append([x_1, x_2])
    all_ys.append(y)

In [7]:
all_xs = np.array(all_xs)
all_ys = np.transpose([all_ys])

In [8]:
sess = tf.Session()

# Merge all the summaries and write them out to /tmp/mnist_logs
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(log_file, sess.graph_def)

init = tf.initialize_all_variables()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [9]:
type(b)

tensorflow.python.ops.variables.Variable

In [10]:
for i in range(steps):
    if datapoint_size == batch_size:
        batch_start_idx = 0
    elif datapoint_size < batch_size:
        raise ValueError("datapoint_size: %d, must be greater than batch_size: %d" % (datapoint_size, batch_size))
    else:
        batch_start_idx = (i * batch_size) % (datapoint_size - batch_size)
  
    batch_end_idx = batch_start_idx + batch_size
    batch_xs = all_xs[batch_start_idx:batch_end_idx]
    batch_ys = all_ys[batch_start_idx:batch_end_idx]
  
    xs = np.array(batch_xs)
    ys = np.array(batch_ys)
    
    all_feed = { x: all_xs, y_: all_ys }
    # Record summary data, and the accuracy every 10 steps
    if i % 10 == 0:
        result = sess.run(merged, feed_dict=all_feed)
        writer.add_summary(result, i)
    else:
        feed = { x: xs, y_: ys }
        sess.run(train_step, feed_dict=feed)
  
    print("After %d iteration:" % i)
    print("W: %s" % sess.run(W))
    print("b : ", sess.run(b))
    print("cost: %f" % sess.run(cost, feed_dict=all_feed))

# NOTE: W should be close to actual_W1, actual_W2, and b should be close to actual_b

After 0 iteration:
W: [[ 0.]
 [ 0.]]
b :  [ 0.]
cost: 1707.340210
After 1 iteration:
W: [[ 0.37759   ]
 [ 0.42454806]]
b :  [ 0.07668001]
cost: 1394.775391
After 2 iteration:
W: [[ 0.71593523]
 [ 0.80933857]]
b :  [ 0.14601459]
cost: 1140.538696
After 3 iteration:
W: [[ 1.01893377]
 [ 1.15825212]]
b :  [ 0.2087269]
cost: 933.715515
After 4 iteration:
W: [[ 1.29009926]
 [ 1.47478759]]
b :  [ 0.2654689]
cost: 765.434875
After 5 iteration:
W: [[ 1.53259885]
 [ 1.76209998]]
b :  [ 0.31682837]
cost: 628.486206
After 6 iteration:
W: [[ 1.74928737]
 [ 2.0230341 ]]
b :  [ 0.36333519]
cost: 517.008911
After 7 iteration:
W: [[ 1.94273794]
 [ 2.26015544]]
b :  [ 0.40546709]
cost: 426.239685
After 8 iteration:
W: [[ 2.11526966]
 [ 2.47577715]]
b :  [ 0.44365478]
cost: 352.306641
After 9 iteration:
W: [[ 2.26897311]
 [ 2.67198539]]
b :  [ 0.4782865]
cost: 292.062561
After 10 iteration:
W: [[ 2.26897311]
 [ 2.67198539]]
b :  [ 0.4782865]
cost: 292.062561
After 11 iteration:
W: [[ 2.40573239]
 [ 2.85

In [11]:
print(sess.run(W))

[[ 2.02913332]
 [ 5.02906275]]


In [12]:
print(sess.run(b))

[ 6.68702221]
